In [395]:
import pandas as pd
import numpy as np
import re

In [396]:
df = pd.read_csv("raw_data/3-Airplane_Crashes_Since_1908.txt")

## dataframe 

In [397]:
df.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


## Easy~Normal 난이도 : 

### 1. 전체 추락 사고 횟수 및 평균 생존율 산출하기

In [360]:
#전체 추락 사고 횟수
df.shape[0] 

5268

In [361]:
# 평균 생존율 
avg_life = (df.Aboard.sum() - df.Fatalities.sum()) / df.Aboard.sum() * 100
avg_life

27.029906399817367

### 2.연 단위로 총 몇 건의 추락 사고가 있었는지, 몇 명이 탑승했는지, 몇 명이 사망했는지, 몇 명이 생존했는지 산출하기

In [362]:
df["Month"] = df["Date"].map(lambda x: x.split('/',3)[0])
df["Day"]   = df["Date"].map(lambda x: x.split('/',3)[1])
df["Year"]  = df["Date"].map(lambda x: x.split('/',3)[2])

#### 연 단위 분석

In [372]:
year_list = df["Year"].unique()
result_year = pd.DataFrame(index = year_list, columns = ["Count","Aboard","Fatalities","Survive"] )
for year in year_list:
    temp = df[df["year"]==year]
    result_year.ix[year]["Count"]  = temp["year"].count()
    result_year.ix[year]["Aboard"] = temp["Aboard"].sum()
    result_year.ix[year]["Fatalities"] = temp["Fatalities"].sum()
    result_year.ix[year]["Survive"] = result_year.ix[year]["Aboard"] - result_year.ix[year]["Fatalities"]

In [373]:
result_year.head()

,Count,Aboard,Fatalities,Survive
1908,1,2,1,1
1912,1,5,5,0
1913,3,51,45,6
1915,2,60,40,20
1916,5,109,108,1


#### 월 단위 분석

In [374]:
month_list = df["Month"].unique()
result_month = pd.DataFrame(index = month_list, columns = ["Count","Aboard","Fatalities","Survive"] )
for month in month_list:
    temp = df[df["Month"] == month]
    result_month.ix[month]["Count"]  = temp["Aboard"].count()
    result_month.ix[month]["Aboard"] = temp["Aboard"].sum()
    result_month.ix[month]["Fatalities"] = temp["Fatalities"].sum()
    result_month.ix[month]["Survive"] = result_month.ix[month]["Aboard"] - result_month.ix[month]["Fatalities"]

In [375]:
result_month.head()

,Count,Aboard,Fatalities,Survive
09,457,13528,10286,3242
07,427,13182,9232,3950
08,472,13953,10174,3779
10,452,10771,8388,2383
03,452,11064,8708,2356


#### 일 단위 분석

In [376]:
day_list = df["Day"].unique()
result_day = pd.DataFrame(index = day_list, columns = ["Count","Aboard","Fatalities","Survive"] )
for day in day_list:
    temp = df[df["Month"] == day]
    result_day.ix[day]["Count"]  = temp["Aboard"].count()
    result_day.ix[day]["Aboard"] = temp["Aboard"].sum()
    result_day.ix[day]["Fatalities"] = temp["Fatalities"].sum()
    result_day.ix[day]["Survive"] = result_day.ix[day]["Aboard"] - result_day.ix[day]["Fatalities"]

In [377]:
result_day.head()

,Count,Aboard,Fatalities,Survive
17,0,0,0,0
12,514,15646,10459,5187
06,385,11607,7909,3698
09,457,13528,10286,3242
05,369,9743,7130,2613


### 3. 추락 사고 횟수 기준 Top 15 Operator, Type, Location 각각 산출하기

#### Top 15 Operator

In [378]:
df["Operator"].value_counts()[:15]

Aeroflot                               179
Military - U.S. Air Force              176
Air France                              70
Deutsche Lufthansa                      65
Air Taxi                                44
China National Aviation Corporation     44
United Air Lines                        44
Military - U.S. Army Air Forces         43
Pan American World Airways              41
US Aerial Mail Service                  36
Military - U.S. Navy                    36
Military - Royal Air Force              36
American Airlines                       36
Indian Airlines                         34
Philippine Air Lines                    33
Name: Operator, dtype: int64

#### Top 15 Type

In [137]:
df["Type"].value_counts()[:15]

Douglas DC-3                                334
de Havilland Canada DHC-6 Twin Otter 300     81
Douglas C-47A                                74
Douglas C-47                                 62
Douglas DC-4                                 40
Yakovlev YAK-40                              37
Antonov AN-26                                36
Junkers JU-52/3m                             32
Douglas C-47B                                29
De Havilland DH-4                            28
Douglas DC-6B                                27
Breguet 14                                   23
Curtiss C-46A                                21
Douglas DC-6                                 20
Douglas C-47-DL                              20
Name: Type, dtype: int64

#### Top 15 Location

In [379]:
df["Location"].value_counts()[:15]

Sao Paulo, Brazil         15
Moscow, Russia            15
Rio de Janeiro, Brazil    14
Manila, Philippines       13
Anchorage, Alaska         13
Bogota, Colombia          13
New York, New York        12
Cairo, Egypt              12
Chicago, Illinois         11
Near Moscow, Russia        9
AtlantiOcean               9
Tehran, Iran               9
Paris, France              8
Denver, Colorado           8
Amsterdam, Netherlands     8
Name: Location, dtype: int64

### 4.추락사고 횟수 기준 Top 3 Operator 별로 가장 많은 사고가 발생한 (연도,월)이 언제였는지 산출하기

In [380]:
Top3_Operator = df["Operator"].value_counts()[:3].index

In [381]:
for operator in Top3_Operator:
    Top_month = df[df["Operator"]==operator]["Month"].value_counts().index[0]
    Top_year = df[df["Operator"]==operator]["Year"].value_counts().index[0]
    print(operator,Top_month,Top_year)

Aeroflot 12 1973
Military - U.S. Air Force 03 1952
Air France 06 1934


### 5.평균 생존율이 높은 Top 100 비행기 Type 도출하기

In [382]:
type_list = df["Type"].unique()
result_type = pd.DataFrame(index = type_list, columns = ["Avg"])
for t in type_list:
    temp = df[df["Type"] == t]
    aboard_temp = temp["Aboard"].sum()
    fatal_temp  = temp["Fatalities"].sum()
    if aboard_temp :
        result_type.ix[t]["Avg"]= (aboard_temp - fatal_temp ) / aboard_temp *100

In [389]:
list(result_type.sort_values(by="Avg",ascending=False).index[:100])

['Airbus A-330-243',
 'Boeing B-747-SP-09',
 'Tupelov Tu-124',
 'Sukhoi Su-27',
 'Bell UH-1B Huey Helicopter',
 'Airbus A-340',
 'Cessna 172',
 'MDonnell Douglas MD-81',
 'Lockheed L-1011',
 'Boeing B-737-7H4',
 'Boeing B-737-3T5',
 'MiG-23',
 'Boeing B-767-233ER',
 'Chance Vought F-8E',
 'Airbus A.320-214',
 'Grummand EA-6B',
 'BAC One-Eleven 528FL',
 'North American F-100D-25NA',
 'Boeing B-747-466',
 'Boeing  377 Stratocruiser',
 'Douglas DC-3 / Cessna 170B',
 'GD F-16D / Lockheed C-130E',
 'Airbus A320-214',
 'F-86 Sabrejet',
 'Boeing 777-236ER',
 'LTVF-8J',
 'Boeing B-747-209BSF',
 'McDonnell Douglas DC-9-51',
 'Boeing B-737-3T0',
 'Boeing B-727-227',
 'Ford 5-AT-D Tri-Motor',
 'Airbus A.330-301',
 'Boeing B-747',
 'Airbus A300B2-101',
 'Lockheed L-1011-200',
 'Boeing B-747-200',
 'Boeing B-707-323B',
 'Convair CV-880-22M-3',
 'Ilyushin IL-18E',
 'Airbus A300-600R',
 'BAC One-Eleven 516FP',
 'McDonnell Douglas MD-90-30',
 'Boeing B-737-297',
 'Sud-Aviation SE210 Caravelle',
 'Airb

### 6.어느 곳에서 비행기가 출발하였을 때 가장 많은 사고가 발생하였는지 산출하기, 어느 곳으로 도착하는 비행기였을 때 가장 많은 사고가 발생하였는지 산출하기

In [428]:
def startend(x):
    # 출발한 도시 이름과 도착한 도시 이름을 찾는 함수
    if pd.isnull(x) :
        return pd.Series({"Departure": np.nan, "Arrival": np.nan}) # 없는 경우 nan 값을 줌
    result = x.split('-')
    if len(result) > 1 :
        return pd.Series({"Departure": result[0].split(',')[0], "Arrival": result[-1].split(',')[0]})
    else :
        return pd.Series({"Departure": np.nan, "Arrival": np.nan})

In [434]:
df[["Arrival","Departure"]] = df["Route"].apply(startend)
# df[["Departure","Arrival"]] 하면 거꾸로 나옴. 왜 그런지는 모르겠다

In [435]:
df["Departure"].value_counts().index[0]

'New York City '

In [436]:
df["Arrival"].value_counts().index[0]

' Paris'

### 7.추락 사고가 가장 많이 발생한 시간대 산출하기 (cf. 불량 데이터 전처리 이슈)

#### 불량 데이터 색출

In [438]:
error_list = []
df_pre = df.dropna(subset = ["Time"]) # time이라는 column 중 na 값 지우기
for i in df_pre["Time"] :
    if not re.match(r"\d{2,2}:\d{2,2}",i) :
        error_list.append(i)
error_list

['c: 1:00',
 'c:17:00',
 'c: 2:00',
 'c:09:00',
 '1:30',
 'c16:50',
 "12'20",
 '18.40',
 'c:09:00',
 '114:20',
 'c14:30',
 '0943',
 '1:00',
 '2:40',
 "22'08",
 'c: 9:40',
 '2:00',
 '8:02',
 '9:30']

In [439]:
def replaceerror(x):
    # 불량 데이터 전처리 함수
    x = re.sub(r"c:?","",x)
    x = re.sub("[',.,\,]","",x)
    x = re.sub(" ","",x)
    if re.match(r"\d{1,1}:\d{2,2}",x) :
        x = '0'+x
    if re.match(r'\d{4,4}',x):
        x = x[0:2] + ':'+x[2:4]
    if re.match(r"\d{3,3}:\d{2,2}",x) :
        x = x[1:]
    return(x)

In [440]:
df_pre["Time"].map(replaceerror).value_counts()

15:00    32
12:00    31
11:00    29
19:30    26
16:00    26
14:00    25
19:00    24
17:00    23
09:30    23
10:30    22
14:30    20
13:00    20
09:00    19
08:30    19
17:30    19
20:30    19
08:00    18
16:30    17
20:00    17
12:30    17
10:15    17
18:00    17
01:00    16
23:00    16
07:00    15
02:00    15
15:30    15
13:30    15
11:30    15
09:40    15
         ..
22:09     1
13:04     1
22:51     1
07:58     1
04:44     1
09:51     1
12:48     1
17:51     1
23:34     1
17:08     1
22:36     1
14:12     1
07:46     1
20:28     1
22:43     1
11:04     1
00:52     1
14:21     1
22:13     1
14:24     1
19:43     1
16:52     1
16:26     1
22:54     1
00:04     1
15:18     1
08:56     1
01:25     1
07:51     1
10:02     1
Name: Time, dtype: int64